<a href="https://colab.research.google.com/github/ArulselvanMadhavan/CSC508/blob/main/A2_ReadingEHRData_creatingPatData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Alzheimer's data intermezzo: creating PatData.csv

Many people are affected by Alzheimers (the Alzheimer’s Association estimates 6 million patients and 11 million unpaid caregivers in the US, see [link](https://www.alz.org/alzheimers-dementia/facts-figures))

Here is some information, written by Dr Raven Baxter [link text](https://threadreaderapp.com/thread/1380339786515423232.html)

Have a look at this short video on [youtube](https://www.youtube.com/watch?v=Eq_Er-tqPsA)

# Having a first look at the data 

The dataset we are using comes from the ADNI project, a huge project to gather data on people with Alzheimer’s disease and healthy controls. 

## Opening the file location and loading libraries

Import libraries

In [1]:
import pandas as pd
from pandas.api.types import CategoricalDtype
import numpy as np
from google.colab import files #For exporting files from google colab notebook

Read the dataset "TADPOLE_D1_D2_57columns.csv" from GitHib

The full repo is here https://github.com/pleunipennings/CSC508Data


In [2]:
url = "https://raw.githubusercontent.com/pleunipennings/CSC508Data/main/TADPOLE_D1_D2_57columns.csv" 
data = pd.read_csv(url)

Look at the first few rows of the data

In [3]:
data.head()

,RID,PTID,VISCODE,SITE,D1,D2,COLPROT,ORIGPROT,EXAMDATE,DX_bl,DXCHANGE,AGE,PTGENDER,PTEDUCAT,PTETHCAT,PTRACCAT,PTMARRY,APOE4,FDG,PIB,AV45,CDRSB,ADAS11,ADAS13,MMSE,RAVLT_immediate,RAVLT_learning,RAVLT_forgetting,RAVLT_perc_forgetting,FAQ,MOCA,EcogPtMem,EcogPtLang,EcogPtVisspat,EcogPtPlan,EcogPtOrgan,EcogPtDivatt,EcogPtTotal,EcogSPMem,EcogSPLang,EcogSPVisspat,EcogSPPlan,EcogSPOrgan,EcogSPDivatt,EcogSPTotal,FLDSTRENG,FSVERSION,Ventricles,Hippocampus,WholeBrain,Entorhinal,Fusiform,MidTemp,ICV,DX,EXAMDATE_bl,CDRSB_bl
0,2,011_S_0002,bl,11,1,1,ADNI1,ADNI1,2005-09-08,CN,1.0,74.3,Male,16,Not Hisp/Latino,White,Married,0.0,1.36926,NaN,NaN,0.0,10.67,18.67,28.0,44.0,4.0,6.0,54.5455,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5 Tesla MRI,Cross-Sectional FreeSurfer (FreeSurfer Version...,118233.0,8336.0,1229740.0,4177.0,16559.0,27936.0,1984660.0,NL,2005-09-08,0.0
1,3,011_S_0003,bl,11,1,0,ADNI1,ADNI1,2005-09-12,AD,3.0,81.3,Male,18,Not Hisp/Latino,White,Married,1.0,1.09079,NaN,NaN,4.5,22.00,31.00,20.0,22.0,1.0,4.0,100.0000,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5 Tesla MRI,Cross-Sectional FreeSurfer (FreeSurfer Version...,84599.0,5319.0,1129830.0,1791.0,15506.0,18422.0,1920690.0,Dementia,2005-09-12,4.5
2,3,011_S_0003,m06,11,1,0,ADNI1,ADNI1,2006-03-13,AD,3.0,81.3,Male,18,Not Hisp/Latino,White,Married,1.0,1.06360,NaN,NaN,6.0,19.00,30.00,24.0,19.0,2.0,6.0,100.0000,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5 Tesla MRI,Cross-Sectional FreeSurfer (FreeSurfer Version...,88580.0,5446.0,1100060.0,2427.0,14400.0,16972.0,1906430.0,Dementia,2005-09-12,4.5
3,3,011_S_0003,m12,11,1,0,ADNI1,ADNI1,2006-09-12,AD,3.0,81.3,Male,18,Not Hisp/Latino,White,Married,1.0,1.10384,NaN,NaN,3.5,24.00,35.00,17.0,31.0,2.0,7.0,100.0000,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5 Tesla MRI,Cross-Sectional FreeSurfer (FreeSurfer Version...,90099.0,5157.0,1095640.0,1596.0,14617.0,17330.0,1903820.0,Dementia,2005-09-12,4.5
4,3,011_S_0003,m24,11,1,0,ADNI1,ADNI1,2007-09-12,AD,3.0,81.3,Male,18,Not Hisp/Latino,White,Married,1.0,1.03871,NaN,NaN,8.0,25.67,37.67,19.0,23.0,1.0,5.0,100.0000,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5 Tesla MRI,Cross-Sectional FreeSurfer (FreeSurfer Version...,97420.0,5139.0,1088560.0,1175.0,14033.0,16398.0,1903420.0,Dementia,2005-09-12,4.5


Find out how big is the data?

In [4]:
data.shape

(12741, 57)

Two columns that are important are **PTID** and **EXAMDATE**. 
PTID is the patient ID and EXAMDATE is –  you guessed it – the date at which the medical exam took place! 

Let's take a look at one patient with the ID: 011_S_0003

In [21]:
data.loc[data['PTID'] == '011_S_0003']

,RID,PTID,VISCODE,SITE,D1,D2,COLPROT,ORIGPROT,EXAMDATE,DX_bl,DXCHANGE,AGE,PTGENDER,PTEDUCAT,PTETHCAT,PTRACCAT,PTMARRY,APOE4,FDG,PIB,AV45,CDRSB,ADAS11,ADAS13,MMSE,RAVLT_immediate,RAVLT_learning,RAVLT_forgetting,RAVLT_perc_forgetting,FAQ,MOCA,EcogPtMem,EcogPtLang,EcogPtVisspat,EcogPtPlan,EcogPtOrgan,EcogPtDivatt,EcogPtTotal,EcogSPMem,EcogSPLang,EcogSPVisspat,EcogSPPlan,EcogSPOrgan,EcogSPDivatt,EcogSPTotal,FLDSTRENG,FSVERSION,Ventricles,Hippocampus,WholeBrain,Entorhinal,Fusiform,MidTemp,ICV,DX,EXAMDATE_bl,CDRSB_bl
1,3,011_S_0003,bl,11,1,0,ADNI1,ADNI1,2005-09-12,AD,3.0,81.3,Male,18,Not Hisp/Latino,White,Married,1.0,1.09079,NaN,NaN,4.5,22.00,31.00,20.0,22.0,1.0,4.0,100.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5 Tesla MRI,Cross-Sectional FreeSurfer (FreeSurfer Version...,84599.0,5319.0,1129830.0,1791.0,15506.0,18422.0,1920690.0,Dementia,2005-09-12,4.5
2,3,011_S_0003,m06,11,1,0,ADNI1,ADNI1,2006-03-13,AD,3.0,81.3,Male,18,Not Hisp/Latino,White,Married,1.0,1.06360,NaN,NaN,6.0,19.00,30.00,24.0,19.0,2.0,6.0,100.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5 Tesla MRI,Cross-Sectional FreeSurfer (FreeSurfer Version...,88580.0,5446.0,1100060.0,2427.0,14400.0,16972.0,1906430.0,Dementia,2005-09-12,4.5
3,3,011_S_0003,m12,11,1,0,ADNI1,ADNI1,2006-09-12,AD,3.0,81.3,Male,18,Not Hisp/Latino,White,Married,1.0,1.10384,NaN,NaN,3.5,24.00,35.00,17.0,31.0,2.0,7.0,100.0,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5 Tesla MRI,Cross-Sectional FreeSurfer (FreeSurfer Version...,90099.0,5157.0,1095640.0,1596.0,14617.0,17330.0,1903820.0,Dementia,2005-09-12,4.5
4,3,011_S_0003,m24,11,1,0,ADNI1,ADNI1,2007-09-12,AD,3.0,81.3,Male,18,Not Hisp/Latino,White,Married,1.0,1.03871,NaN,NaN,8.0,25.67,37.67,19.0,23.0,1.0,5.0,100.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5 Tesla MRI,Cross-Sectional FreeSurfer (FreeSurfer Version...,97420.0,5139.0,1088560.0,1175.0,14033.0,16398.0,1903420.0,Dementia,2005-09-12,4.5
5732,3,011_S_0003,m18,11,1,0,ADNI1,ADNI1,2007-03-12,AD,NaN,81.3,Male,18,Not Hisp/Latino,White,Married,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005-09-12,4.5


## Assignment 1: Looking at the data

In which years did this patient come in for exams?  
What is the age, marital status and race of the patient?


- 2005, 2006, 2007
- Age: 81
- Married
- White

# Categoricals and Ordered Categories! 


Some examples of categorical data are gender, social class, blood type, or rating scale.

Here, we're going to make the column Diagnosis 'DX' into a categorical data type and specify an order on the categories. We need to do this so that we can use the ordering to select the data we need. 

Here are what the abbreviations are:
*   **NL** = no Alzheimers (cognitive normal)
*   **MCI** = Mild Cognitive Impairement
*   **Dementia** = has Alzheimers

Here is a guide for Categorical data types: https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html#categoricaldtype


In [12]:
data['DX']

0              NL
1        Dementia
2        Dementia
3        Dementia
4        Dementia
           ...   
12736         NaN
12737         NaN
12738         NaN
12739         MCI
12740         MCI
Name: DX, Length: 12741, dtype: category
Categories (8, object): ['NL' < 'NL to MCI' < 'MCI to NL' < 'MCI' < 'Dementia to MCI' <
                         'MCI to Dementia' < 'Dementia' < 'NL to Dementia']

In [13]:
DX_type = CategoricalDtype(categories=['NL', 'NL to MCI', 'MCI to NL', 'MCI', 
                           'Dementia to MCI', 'MCI to Dementia', 
                           'Dementia', 'NL to Dementia'], ordered=True)
data['DX'] = data['DX'].astype(DX_type)

## Assignment 2: Ordered Categories

Create ordered categories for the column 'DX_bl' (diagnosis at baseline). The levels are:
*   **CN**: cognitive normal
*   **SMC**: subjective memory concerns
*   **EMCI**: Early Mild Cognitive Impairment
*   **LMCI**: Late Mild Cognitive Impairment
*   **AD**: Alzheimer’s Disease

Check that your code worked. 
Take a screenshot of your code and include it in the pdf or wordfile you submit on iLearn. 


In [28]:
# Put your code here and take a screenshot to include in your word file to hand in on iLearn. 
dbl = data['DX_bl'].map(lambda d: d)
dbl_uniq = pd.unique(dbl)
dbl_type = CategoricalDtype(categories=['CN','SMC','EMCI','LMCI','AD'], ordered=True)
data['DX_bl'] = data['DX_bl'].astype(dbl_type)
data['DX_bl']

0          CN
1          AD
2          AD
3          AD
4          AD
         ... 
12736    LMCI
12737    EMCI
12738    EMCI
12739    EMCI
12740    EMCI
Name: DX_bl, Length: 12741, dtype: category
Categories (5, object): ['CN' < 'SMC' < 'EMCI' < 'LMCI' < 'AD']

# Grouping data: One row per patient

For some types of analysis, it is useful to have just one row per patient instead of multiple. 

Let's create a dataset that has just one row per patient. To do so, we need to 'group' the data based on something unique. In this dataset, we can use the Patient ID and group all the entries that have the same ID. Then we can aggregate (this really mean combine the data) and grab the max entry or calculate the mean for the groupings that are the same. 

Note: Real-life data is always messy and may be missing. So we want to make sure we use the pandas dropna argument and set it to False, otherwise all entries with 'NA's will be removed. You'll actually be missing important patient data!

Guide for Summarising, Aggregating, Grouping data in Python: https://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/

Guide for handling missing data: https://machinelearningmastery.com/handle-missing-data-python/

In [30]:
Pat_Data = data.groupby(
    ['PTID', 'PTGENDER', 'PTEDUCAT','PTETHCAT', 'PTRACCAT','PTMARRY','APOE4'], dropna=False
    ).aggregate(
        {'DX': 'max',
         'AGE': 'median',
         'Ventricles': 'mean',
         'Hippocampus': 'mean',
         'WholeBrain': 'mean', 
         'Entorhinal': 'mean', 
         'Fusiform': 'mean', 
         'MidTemp': 'mean', 
         'ICV': 'mean'}
          ).reset_index()

In [31]:
Pat_Data.shape

(1737, 16)

#Assigment 3: shape of the dataframe
How did the shape of the dataframe change? How many rows did the original datafram have per patient approximately? 

In [32]:
data.shape[0]/Pat_Data.shape[0]

7.335060449050086

7 rows per patient

#More categorical variables

What if we had multiple columns that were categorical? Here, we can make multiple columns into categoricals by using the lambda function. A lambda function can take any number of arguments, but can only have one expression.


Here are some guides on the lambda function and the apply function
*   lambda function: https://www.w3schools.com/python/python_lambda.asp

*   apply function: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html?highlight=apply






Now, we can take a look at the data type of each column. We can check to see if the columns we have changed (DX, DX_bl, PTGENDER, PTETHCAT, PTRACCAT, and PTMARRY) are correct and listed as 'category'. 

In [33]:
cols = ['PTGENDER', 'PTETHCAT', 'PTRACCAT', 'PTMARRY']
data[cols] = data[cols].apply(lambda x: x.astype('category'))

In [ ]:
data.dtypes

RID                         int64
PTID                       object
VISCODE                    object
SITE                        int64
D1                          int64
D2                          int64
COLPROT                    object
ORIGPROT                   object
EXAMDATE                   object
DX_bl                      object
DXCHANGE                  float64
AGE                       float64
PTGENDER                 category
PTEDUCAT                    int64
PTETHCAT                 category
PTRACCAT                 category
PTMARRY                  category
APOE4                     float64
FDG                       float64
PIB                       float64
AV45                      float64
CDRSB                     float64
ADAS11                    float64
ADAS13                    float64
MMSE                      float64
RAVLT_immediate           float64
RAVLT_learning            float64
RAVLT_forgetting          float64
RAVLT_perc_forgetting     float64
FAQ           

# Summary Statistics

You may be curious about the patients in the study. By using the describe() function, you could take a look at the summary statistics. In this case, we see that there are more males in the study.

*   count = the number of elements in your dataset
*   unique = the total number of unique elements
*   top = most frequent entry
*   freq = Frequency of the most frequent entry

Depending on the column data type, the summary statistics could also include the mean, std, min max, etc

Guide on summary statistics: https://www.askpython.com/python/examples/calculate-summary-statistics

What happens if you want to know the count of all of the cateogories? You can use the value_counts() function. Take a look at the two code chunks below.

In [34]:
Pat_Data['PTGENDER'].describe()

count     1737
unique       2
top       Male
freq       957
Name: PTGENDER, dtype: object

In [35]:
Pat_Data['PTGENDER'].value_counts()

Male      957
Female    780
Name: PTGENDER, dtype: int64

## Assignment 4: Summary

After you created PatData, use head() to have a look at the first rows.  

1.   Look at the summary statistics for race ('PTRACCAT'). What is the most frequent answer? Use describe() and value_counts(). Which function do you like best? 
2.   Try Pat_Data.describe() and look at the summary statistics for the whole dataset. What is the average age? What is the age of the youngest patient? What is the age of the oldest patient? Does it show all variables? 





In [36]:
Pat_Data.head()

,PTID,PTGENDER,PTEDUCAT,PTETHCAT,PTRACCAT,PTMARRY,APOE4,DX,AGE,Ventricles,Hippocampus,WholeBrain,Entorhinal,Fusiform,MidTemp,ICV
0,002_S_0295,Male,18,Not Hisp/Latino,White,Married,1.0,NL,84.8,43332.500000,6805.125000,1.071568e+06,3752.625000,17693.875000,19420.125000,1.649602e+06
1,002_S_0413,Female,16,Not Hisp/Latino,White,Married,0.0,NL,76.3,31936.454545,6824.636364,1.055413e+06,4131.090909,20095.909091,20235.545455,1.600009e+06
2,002_S_0559,Male,16,Not Hisp/Latino,White,Widowed,1.0,NL,79.3,38410.666667,7496.666667,1.092807e+06,3998.333333,18993.000000,22226.000000,1.703968e+06
3,002_S_0619,Male,12,Not Hisp/Latino,White,Married,2.0,Dementia,77.5,120529.500000,5812.000000,1.093932e+06,2773.000000,20675.000000,19959.000000,2.070530e+06
4,002_S_0685,Female,16,Not Hisp/Latino,White,Married,0.0,NL,89.6,40921.571429,7063.250000,9.800458e+05,3894.375000,14152.250000,18133.625000,1.521331e+06


In [38]:
Pat_Data['PTRACCAT'].describe()

count      1737
unique        7
top       White
freq       1605
Name: PTRACCAT, dtype: object

In [39]:
Pat_Data['PTRACCAT'].value_counts()

White                1605
Black                  77
Asian                  29
More than one          18
Unknown                 3
Am Indian/Alaskan       3
Hawaiian/Other PI       2
Name: PTRACCAT, dtype: int64

The most frequent answer is white. I like the function describe since it gives useful high level stats

# Exporting as a csv file

Save the newly created dataset that contains one row per patient.
This will be saved onto your Desktop.
Replace "YourVersion" by your own name so that you know that this file was made by you. 

In [41]:
Pat_Data.to_csv('PatData_Arul.csv')
files.download('PatData_Arul.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>